# V4 of GUI worked well and displayed logging of 4 ports, while we needed to add some more features to it such as capability of saving the logged data, etc. In this version I am trying to include those features in the GUI.

# The code below works, however fails to save the data from ports in CSV files

In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import re
import tkinter as tk
from tkinter import ttk
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device 
gpio_pins_per_device = {
    'Port 1': {
        "LeftPoke": 17,
        "RightPoke": 27,
        "Pellet": 22,
    },
    'Port 2': {
        "LeftPoke": 10,
        "RightPoke": 9,
        "Pellet": 11,
    },
    'Port 3': {
        "LeftPoke": 0,
        "RightPoke": 5,
        "Pellet": 6,
    },
    'Port 4': {
        "LeftPoke": 13,
        "RightPoke": 19,
        "Pellet": 26,
    },
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Create a lock for thread-safe access to shared variables
pellet_lock = threading.Lock()

# Track the state of Pellet in Well per port_identifier
pellet_in_well = {}

# Global event to signal threads to stop
stop_event = threading.Event()

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] Processing event: {event_type}"
    q.put(message)

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"Left poke event triggered.")
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"Right poke event triggered.")
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"Left poke with pellet, signal triggered.")
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"Right poke with pellet, signal triggered.")
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

# Function to get device mappings based on physical USB ports
def get_device_mappings_by_usb_port():
    device_mappings = []
    usb_port_mapping = {
        'usb-0:1.1': 'Port 1',
        'usb-0:1.2': 'Port 2',
        'usb-0:1.3': 'Port 3',
        'usb-0:1.4': 'Port 4',
    }
    for symlink in os.listdir('/dev/serial/by-path/'):
        symlink_path = os.path.join('/dev/serial/by-path/', symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if not usb_port_path:
                continue
            port_identifier = usb_port_mapping.get(usb_port_path)
            if port_identifier:
                device_mappings.append({
                    'serial_port': serial_port,
                    'port_identifier': port_identifier,
                })
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port, port_identifier, gpio_pins, q):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        logging.info(f"Opened serial port {serial_port} for {port_identifier}")
        q.put("Ready")
    except serial.SerialException as e:
        q.put(f"Error opening serial port: {e}")
        return
    try:
        while not stop_event.is_set():
            line = ser.readline().decode('utf-8').strip()
            if line:
                data_list = line.split(",")
                if len(data_list) >= 10:
                    event_type = data_list[9]
                    process_event(event_type, port_identifier, gpio_pins, q)
    finally:
        ser.close()

# GUI Application Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("FED3 Data Monitor")

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()

        # Set up the mainframe
        self.mainframe = ttk.Frame(self.root, padding="3 3 12 12")
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.mainframe.columnconfigure(0, weight=1)
        self.mainframe.rowconfigure(0, weight=1)

        # Create GUI for each port
        self.setup_ports()

        # Add fields for experimenter and experiment name
        self.create_controls()

        # Start the periodic GUI update function
        self.root.after(100, self.update_gui)

    def setup_ports(self):
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            frame = ttk.LabelFrame(self.mainframe, text=port_name)
            frame.grid(column=idx, row=0, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=40, height=20)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

            self.port_widgets[port_name] = {
                'frame': frame,
                'status_label': status_label,
                'text_widget': text_widget
            }
            self.port_queues[port_name] = queue.Queue()

        for idx in range(len(port_names)):
            self.mainframe.columnconfigure(idx, weight=1)

    def create_controls(self):
        # Experimenter name
        ttk.Label(self.mainframe, text="Experimenter Name:").grid(column=0, row=2, padx=5, pady=5)
        self.experimenter_entry = ttk.Entry(self.mainframe, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=2, padx=5, pady=5)

        # Experiment name
        ttk.Label(self.mainframe, text="Experiment Name:").grid(column=2, row=2, padx=5, pady=5)
        self.experiment_entry = ttk.Entry(self.mainframe, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=2, padx=5, pady=5)

        # Start button
        self.start_button = ttk.Button(self.mainframe, text="Start", command=self.start_experiment)
        self.start_button.grid(column=0, row=3, padx=5, pady=10)

        # Stop button
        self.stop_button = ttk.Button(self.mainframe, text="Stop", command=self.stop_experiment)
        self.stop_button.grid(column=1, row=3, padx=5, pady=10)

    def start_experiment(self):
        # Create folder structure for saving files
        experimenter_folder = f"./{self.experimenter_name.get()}"
        experiment_folder = f"{experimenter_folder}/{self.experiment_name.get()}"
        os.makedirs(experiment_folder, exist_ok=True)

        # Start reading from the devices
        device_mappings = get_device_mappings_by_usb_port()
        threads = []
        for mapping in device_mappings:
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            if not gpio_pins:
                continue
            q = self.port_queues[port_identifier]
            t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q))
            t.daemon = True
            t.start()
            threads.append(t)

    def update_gui(self):
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass
        self.root.after(100, self.update_gui)

    def stop_experiment(self):
        stop_event.set()
        # Save logs into CSV
        for port_identifier in self.port_queues:
            self.save_to_csv(port_identifier)
        self.root.quit()  # Close the GUI properly

    def save_to_csv(self, port_identifier):
        experimenter_folder = f"./{self.experimenter_name.get()}"
        experiment_folder = f"{experimenter_folder}/{self.experiment_name.get()}"
        os.makedirs(experiment_folder, exist_ok=True)
        
        filename = f"{experiment_folder}/{port_identifier}.csv"
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Timestamp", "Event"])
            while not self.port_queues[port_identifier].empty():
                writer.writerow([datetime.datetime.now(), self.port_queues[port_identifier].get()])

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


# code below trying to figure out why GUI does not save the data and quit after pressing STOP

In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import re
import tkinter as tk
from tkinter import ttk
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device 
gpio_pins_per_device = {
    'Port 1': {
        "LeftPoke": 17,
        "RightPoke": 27,
        "Pellet": 22,
    },
    'Port 2': {
        "LeftPoke": 10,
        "RightPoke": 9,
        "Pellet": 11,
    },
    'Port 3': {
        "LeftPoke": 0,
        "RightPoke": 5,
        "Pellet": 6,
    },
    'Port 4': {
        "LeftPoke": 13,
        "RightPoke": 19,
        "Pellet": 26,
    },
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Create a lock for thread-safe access to shared variables
pellet_lock = threading.Lock()

# Track the state of Pellet in Well per port_identifier
pellet_in_well = {}

# Global event to signal threads to stop
stop_event = threading.Event()

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke event triggered.")
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke event triggered.")
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke with pellet, signal triggered.")
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke with pellet, signal triggered.")
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

# Function to get device mappings based on physical USB ports
def get_device_mappings_by_usb_port():
    device_mappings = []
    usb_port_mapping = {
        'usb-0:1.1': 'Port 1',
        'usb-0:1.2': 'Port 2',
        'usb-0:1.3': 'Port 3',
        'usb-0:1.4': 'Port 4',
    }
    for symlink in os.listdir('/dev/serial/by-path/'):
        symlink_path = os.path.join('/dev/serial/by-path/', symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if not usb_port_path:
                continue
            port_identifier = usb_port_mapping.get(usb_port_path)
            if port_identifier:
                device_mappings.append({
                    'serial_port': serial_port,
                    'port_identifier': port_identifier,
                })
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port, port_identifier, gpio_pins, q):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        logging.info(f"Opened serial port {serial_port} for {port_identifier}")
        q.put("Ready")
    except serial.SerialException as e:
        q.put(f"Error opening serial port: {e}")
        return
    try:
        while not stop_event.is_set():
            line = ser.readline().decode('utf-8').strip()
            if line:
                data_list = line.split(",")
                q.put(f"{port_identifier} raw data: {data_list}")
                if len(data_list) >= 10:
                    event_type = data_list[9]
                    process_event(event_type, port_identifier, gpio_pins, q)
    finally:
        ser.close()

# GUI Application Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("FED3 Data Monitor")

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()

        # Set up the mainframe
        self.mainframe = ttk.Frame(self.root, padding="3 3 12 12")
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.mainframe.columnconfigure(0, weight=1)
        self.mainframe.rowconfigure(0, weight=1)

        # Create GUI for each port
        self.setup_ports()

        # Add fields for experimenter and experiment name
        self.create_controls()

        # Start the periodic GUI update function
        self.root.after(100, self.update_gui)

    def setup_ports(self):
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            frame = ttk.LabelFrame(self.mainframe, text=port_name)
            frame.grid(column=idx, row=0, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=40, height=20)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

            self.port_widgets[port_name] = {
                'frame': frame,
                'status_label': status_label,
                'text_widget': text_widget
            }
            self.port_queues[port_name] = queue.Queue()

        for idx in range(len(port_names)):
            self.mainframe.columnconfigure(idx, weight=1)

    def create_controls(self):
        # Experimenter name
        ttk.Label(self.mainframe, text="Experimenter Name:").grid(column=0, row=2, padx=5, pady=5)
        self.experimenter_entry = ttk.Entry(self.mainframe, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=2, padx=5, pady=5)

        # Experiment name
        ttk.Label(self.mainframe, text="Experiment Name:").grid(column=2, row=2, padx=5, pady=5)
        self.experiment_entry = ttk.Entry(self.mainframe, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=2, padx=5, pady=5)

        # Start button
        self.start_button = ttk.Button(self.mainframe, text="Start", command=self.start_experiment)
        self.start_button.grid(column=0, row=3, padx=5, pady=10)

        # Stop button
        self.stop_button = ttk.Button(self.mainframe, text="Stop", command=self.stop_experiment)
        self.stop_button.grid(column=1, row=3, padx=5, pady=10)

    def start_experiment(self):
        # Create folder structure for saving files
        experimenter_folder = f"./{self.experimenter_name.get()}"
        experiment_folder = f"{experimenter_folder}/{self.experiment_name.get()}"
        os.makedirs(experiment_folder, exist_ok=True)

        # Start reading from the devices
        self.device_mappings = get_device_mappings_by_usb_port()
        self.threads = []
        for mapping in self.device_mappings:
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            if not gpio_pins:
                continue
            q = self.port_queues[port_identifier]
            t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q))
            t.daemon = True
            t.start()
            self.threads.append(t)

    def update_gui(self):
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass
        self.root.after(100, self.update_gui)

    def stop_experiment(self):
        stop_event.set()
        # Save logs into CSV
        for port_identifier in self.port_queues:
            self.save_to_csv(port_identifier)
        
        # Clean up threads and close the window
        for t in self.threads:
            t.join()  # Ensure all threads are properly stopped
        GPIO.cleanup()
        self.root.quit()  # Close the GUI window properly

    def save_to_csv(self, port_identifier):
        experimenter_folder = f"./{self.experimenter_name.get()}"
        experiment_folder = f"{experimenter_folder}/{self.experiment_name.get()}"
        os.makedirs(experiment_folder, exist_ok=True)
        
        filename = f"{experiment_folder}/{port_identifier}.csv"
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Timestamp", "Event"])
            while not self.port_queues[port_identifier].empty():
                writer.writerow([datetime.datetime.now(), self.port_queues[port_identifier].get()])

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


# code above created folders, but does not log data on csv fiels

In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import re
import tkinter as tk
from tkinter import ttk
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device 
gpio_pins_per_device = {
    'Port 1': {
        "LeftPoke": 17,
        "RightPoke": 27,
        "Pellet": 22,
    },
    'Port 2': {
        "LeftPoke": 10,
        "RightPoke": 9,
        "Pellet": 11,
    },
    'Port 3': {
        "LeftPoke": 0,
        "RightPoke": 5,
        "Pellet": 6,
    },
    'Port 4': {
        "LeftPoke": 13,
        "RightPoke": 19,
        "Pellet": 26,
    },
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Create a lock for thread-safe access to shared variables
pellet_lock = threading.Lock()

# Track the state of Pellet in Well per port_identifier
pellet_in_well = {}

# Global event to signal threads to stop
stop_event = threading.Event()

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke event triggered.")
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke event triggered.")
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke with pellet, signal triggered.")
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke with pellet, signal triggered.")
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

# Function to get device mappings based on physical USB ports
def get_device_mappings_by_usb_port():
    device_mappings = []
    usb_port_mapping = {
        'usb-0:1.1': 'Port 1',
        'usb-0:1.2': 'Port 2',
        'usb-0:1.3': 'Port 3',
        'usb-0:1.4': 'Port 4',
    }
    for symlink in os.listdir('/dev/serial/by-path/'):
        symlink_path = os.path.join('/dev/serial/by-path/', symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if not usb_port_path:
                continue
            port_identifier = usb_port_mapping.get(usb_port_path)
            if port_identifier:
                device_mappings.append({
                    'serial_port': serial_port,
                    'port_identifier': port_identifier,
                })
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port, port_identifier, gpio_pins, q):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        logging.info(f"Opened serial port {serial_port} for {port_identifier}")
        q.put("Ready")
    except serial.SerialException as e:
        q.put(f"Error opening serial port: {e}")
        return
    try:
        while not stop_event.is_set():
            line = ser.readline().decode('utf-8').strip()
            if line:
                data_list = line.split(",")
                q.put(f"{port_identifier} raw data: {data_list}")
                if len(data_list) >= 10:
                    event_type = data_list[9]
                    process_event(event_type, port_identifier, gpio_pins, q)
    finally:
        ser.close()

# GUI Application Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("FED3 Data Monitor")

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()

        # Set up the mainframe
        self.mainframe = ttk.Frame(self.root, padding="3 3 12 12")
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.mainframe.columnconfigure(0, weight=1)
        self.mainframe.rowconfigure(0, weight=1)

        # Create GUI for each port
        self.setup_ports()

        # Add fields for experimenter and experiment name
        self.create_controls()

        # Start the periodic GUI update function
        self.root.after(100, self.update_gui)

    def setup_ports(self):
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            frame = ttk.LabelFrame(self.mainframe, text=port_name)
            frame.grid(column=idx, row=0, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=40, height=20)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

            self.port_widgets[port_name] = {
                'frame': frame,
                'status_label': status_label,
                'text_widget': text_widget
            }
            self.port_queues[port_name] = queue.Queue()

        for idx in range(len(port_names)):
            self.mainframe.columnconfigure(idx, weight=1)

    def create_controls(self):
        # Experimenter name
        ttk.Label(self.mainframe, text="Experimenter Name:").grid(column=0, row=2, padx=5, pady=5)
        self.experimenter_entry = ttk.Entry(self.mainframe, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=2, padx=5, pady=5)

        # Experiment name
        ttk.Label(self.mainframe, text="Experiment Name:").grid(column=2, row=2, padx=5, pady=5)
        self.experiment_entry = ttk.Entry(self.mainframe, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=2, padx=5, pady=5)

        # Start button
        self.start_button = ttk.Button(self.mainframe, text="Start", command=self.start_experiment)
        self.start_button.grid(column=0, row=3, padx=5, pady=10)

        # Stop button
        self.stop_button = ttk.Button(self.mainframe, text="Stop", command=self.stop_experiment)
        self.stop_button.grid(column=1, row=3, padx=5, pady=10)

    def start_experiment(self):
        # Create folder structure for saving files
        experimenter_folder = f"./{self.experimenter_name.get()}"
        experiment_folder = f"{experimenter_folder}/{self.experiment_name.get()}"
        os.makedirs(experiment_folder, exist_ok=True)

        # Start reading from the devices
        self.device_mappings = get_device_mappings_by_usb_port()
        self.threads = []
        for mapping in self.device_mappings:
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            if not gpio_pins:
                continue
            q = self.port_queues[port_identifier]
            t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q))
            t.daemon = True
            t.start()
            self.threads.append(t)

    def update_gui(self):
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass
        self.root.after(100, self.update_gui)

    def stop_experiment(self):
        stop_event.set()
        # Save logs into CSV
        for port_identifier in self.port_queues:
            self.save_to_csv(port_identifier)
        
        # Clean up threads and close the window
        for t in self.threads:
            t.join()  # Ensure all threads are properly stopped
        GPIO.cleanup()
        self.root.quit()  # Close the GUI window properly

    def save_to_csv(self, port_identifier):
        experimenter_folder = f"./{self.experimenter_name.get()}"
        experiment_folder = f"{experimenter_folder}/{self.experiment_name.get()}"
        os.makedirs(experiment_folder, exist_ok=True)
        
        filename = f"{experiment_folder}/{port_identifier}.csv"
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Timestamp", "Event"])
            while not self.port_queues[port_identifier].empty():
                writer.writerow([datetime.datetime.now(), self.port_queues[port_identifier].get()])

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


# code above also shows the raw data in panels of GUI but does not properly save it in CSV files. also STOP button does not close the window

In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import re
import tkinter as tk
from tkinter import ttk
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device 
gpio_pins_per_device = {
    'Port 1': {
        "LeftPoke": 17,
        "RightPoke": 27,
        "Pellet": 22,
    },
    'Port 2': {
        "LeftPoke": 10,
        "RightPoke": 9,
        "Pellet": 11,
    },
    'Port 3': {
        "LeftPoke": 0,
        "RightPoke": 5,
        "Pellet": 6,
    },
    'Port 4': {
        "LeftPoke": 13,
        "RightPoke": 19,
        "Pellet": 26,
    },
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Create a lock for thread-safe access to shared variables
pellet_lock = threading.Lock()

# Track the state of Pellet in Well per port_identifier
pellet_in_well = {}

# Global event to signal threads to stop
stop_event = threading.Event()

# Define the column headers for CSV
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke event triggered.")
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke event triggered.")
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke with pellet, signal triggered.")
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke with pellet, signal triggered.")
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

# Function to get device mappings based on physical USB ports
def get_device_mappings_by_usb_port():
    device_mappings = []
    usb_port_mapping = {
        'usb-0:1.1': 'Port 1',
        'usb-0:1.2': 'Port 2',
        'usb-0:1.3': 'Port 3',
        'usb-0:1.4': 'Port 4',
    }
    for symlink in os.listdir('/dev/serial/by-path/'):
        symlink_path = os.path.join('/dev/serial/by-path/', symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if not usb_port_path:
                continue
            port_identifier = usb_port_mapping.get(usb_port_path)
            if port_identifier:
                device_mappings.append({
                    'serial_port': serial_port,
                    'port_identifier': port_identifier,
                })
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port, port_identifier, gpio_pins, q):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        logging.info(f"Opened serial port {serial_port} for {port_identifier}")
        q.put("Ready")
    except serial.SerialException as e:
        q.put(f"Error opening serial port: {e}")
        return
    try:
        while not stop_event.is_set():
            line = ser.readline().decode('utf-8').strip()
            if line:
                data_list = line.split(",")
                q.put(f"{port_identifier} raw data: {data_list}")
                if len(data_list) >= 10:
                    event_type = data_list[9]
                    # Log system time instead of FED3 time
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                    data_list[0] = timestamp  # Replace FED3 timestamp with system timestamp
                    process_event(event_type, port_identifier, gpio_pins, q)
                    # Log full data with system time
                    q.put(f"Data logged: {data_list}")
                    q.put(data_list)
    finally:
        ser.close()

# GUI Application Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("FED3 Data Monitor")

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()

        # Set up the mainframe
        self.mainframe = ttk.Frame(self.root, padding="3 3 12 12")
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.mainframe.columnconfigure(0, weight=1)
        self.mainframe.rowconfigure(0, weight=1)

        # Create GUI for each port
        self.setup_ports()

        # Add fields for experimenter and experiment name
        self.create_controls()

        # Start the periodic GUI update function
        self.root.after(100, self.update_gui)

    def setup_ports(self):
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            frame = ttk.LabelFrame(self.mainframe, text=port_name)
            frame.grid(column=idx, row=0, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=40, height=20)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

            self.port_widgets[port_name] = {
                'frame': frame,
                'status_label': status_label,
                'text_widget': text_widget
            }
            self.port_queues[port_name] = queue.Queue()

        for idx in range(len(port_names)):
            self.mainframe.columnconfigure(idx, weight=1)

    def create_controls(self):
        # Experimenter name
        ttk.Label(self.mainframe, text="Experimenter Name:").grid(column=0, row=2, padx=5, pady=5)
        self.experimenter_entry = ttk.Entry(self.mainframe, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=2, padx=5, pady=5)

        # Experiment name
        ttk.Label(self.mainframe, text="Experiment Name:").grid(column=2, row=2, padx=5, pady=5)
        self.experiment_entry = ttk.Entry(self.mainframe, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=2, padx=5, pady=5)

        # Start button
        self.start_button = ttk.Button(self.mainframe, text="Start", command=self.start_experiment)
        self.start_button.grid(column=0, row=3, padx=5, pady=10)

        # Stop button
        self.stop_button = ttk.Button(self.mainframe, text="Stop", command=self.stop_experiment)
        self.stop_button.grid(column=1, row=3, padx=5, pady=10)

    def start_experiment(self):
        # Create folder structure for saving files
        experimenter_folder = f"./{self.experimenter_name.get()}"
        experiment_folder = f"{experimenter_folder}/{self.experiment_name.get()}"
        os.makedirs(experiment_folder, exist_ok=True)

        # Start reading from the devices
        self.device_mappings = get_device_mappings_by_usb_port()
        self.threads = []
        for mapping in self.device_mappings:
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            if not gpio_pins:
                continue
            q = self.port_queues[port_identifier]
            t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q))
            t.daemon = True
            t.start()
            self.threads.append(t)

    def update_gui(self):
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if isinstance(message, list):
                        # This is a data list from the FED3 device, save it for logging
                        self.save_to_csv(port_identifier, message)
                    elif message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass
        self.root.after(100, self.update_gui)

    def stop_experiment(self):
        stop_event.set()
        # Clean up threads and close the window
        for t in self.threads:
            t.join()  # Ensure all threads are properly stopped
        GPIO.cleanup()
        self.root.quit()  # Close the GUI window properly

    def save_to_csv(self, port_identifier, data_row):
        experimenter_folder = f"./{self.experimenter_name.get()}"
        experiment_folder = f"{experimenter_folder}/{self.experiment_name.get()}"
        os.makedirs(experiment_folder, exist_ok=True)
        
        filename = f"{experiment_folder}/{port_identifier}.csv"
        file_exists = os.path.isfile(filename)
        with open(filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            if not file_exists:
                writer.writerow(column_headers)  # Write header only once
            writer.writerow(data_row)

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


# code above works fine and saves the data as well, however STOP button does not function properly 

In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import re
import tkinter as tk
from tkinter import ttk
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device 
gpio_pins_per_device = {
    'Port 1': {
        "LeftPoke": 17,
        "RightPoke": 27,
        "Pellet": 22,
    },
    'Port 2': {
        "LeftPoke": 10,
        "RightPoke": 9,
        "Pellet": 11,
    },
    'Port 3': {
        "LeftPoke": 0,
        "RightPoke": 5,
        "Pellet": 6,
    },
    'Port 4': {
        "LeftPoke": 13,
        "RightPoke": 19,
        "Pellet": 26,
    },
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Create a lock for thread-safe access to shared variables
pellet_lock = threading.Lock()

# Track the state of Pellet in Well per port_identifier
pellet_in_well = {}

# Global event to signal threads to stop
stop_event = threading.Event()

# Define the column headers for CSV
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke event triggered.")
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke event triggered.")
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke with pellet, signal triggered.")
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke with pellet, signal triggered.")
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

# Function to get device mappings based on physical USB ports
def get_device_mappings_by_usb_port():
    device_mappings = []
    usb_port_mapping = {
        'usb-0:1.1': 'Port 1',
        'usb-0:1.2': 'Port 2',
        'usb-0:1.3': 'Port 3',
        'usb-0:1.4': 'Port 4',
    }
    for symlink in os.listdir('/dev/serial/by-path/'):
        symlink_path = os.path.join('/dev/serial/by-path/', symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if not usb_port_path:
                continue
            port_identifier = usb_port_mapping.get(usb_port_path)
            if port_identifier:
                device_mappings.append({
                    'serial_port': serial_port,
                    'port_identifier': port_identifier,
                })
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port, port_identifier, gpio_pins, q):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        logging.info(f"Opened serial port {serial_port} for {port_identifier}")
        q.put("Ready")
    except serial.SerialException as e:
        q.put(f"Error opening serial port: {e}")
        return
    try:
        while not stop_event.is_set():
            line = ser.readline().decode('utf-8').strip()
            if line:
                data_list = line.split(",")
                q.put(f"{port_identifier} raw data: {data_list}")
                if len(data_list) >= 10:
                    event_type = data_list[9]
                    # Log system time instead of FED3 time
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                    data_list[0] = timestamp  # Replace FED3 timestamp with system timestamp
                    process_event(event_type, port_identifier, gpio_pins, q)
                    # Log full data with system time
                    q.put(f"Data logged: {data_list}")
                    q.put(data_list)
    finally:
        ser.close()

# GUI Application Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("FED3 Data Monitor")

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()

        # Set up the mainframe
        self.mainframe = ttk.Frame(self.root, padding="3 3 12 12")
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.mainframe.columnconfigure(0, weight=1)
        self.mainframe.rowconfigure(0, weight=1)

        # Create GUI for each port
        self.setup_ports()

        # Add fields for experimenter and experiment name
        self.create_controls()

        # Start the periodic GUI update function
        self.root.after(100, self.update_gui)

    def setup_ports(self):
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            frame = ttk.LabelFrame(self.mainframe, text=port_name)
            frame.grid(column=idx, row=0, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=40, height=20)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

            self.port_widgets[port_name] = {
                'frame': frame,
                'status_label': status_label,
                'text_widget': text_widget
            }
            self.port_queues[port_name] = queue.Queue()

        for idx in range(len(port_names)):
            self.mainframe.columnconfigure(idx, weight=1)

    def create_controls(self):
        # Experimenter name
        ttk.Label(self.mainframe, text="Experimenter Name:").grid(column=0, row=2, padx=5, pady=5)
        self.experimenter_entry = ttk.Entry(self.mainframe, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=2, padx=5, pady=5)

        # Experiment name
        ttk.Label(self.mainframe, text="Experiment Name:").grid(column=2, row=2, padx=5, pady=5)
        self.experiment_entry = ttk.Entry(self.mainframe, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=2, padx=5, pady=5)

        # Start button
        self.start_button = ttk.Button(self.mainframe, text="Start", command=self.start_experiment)
        self.start_button.grid(column=0, row=3, padx=5, pady=10)

        # Stop button
        self.stop_button = ttk.Button(self.mainframe, text="Stop", command=self.stop_experiment)
        self.stop_button.grid(column=1, row=3, padx=5, pady=10)

    def start_experiment(self):
        # Create folder structure for saving files
        experimenter_folder = f"./{self.experimenter_name.get()}"
        experiment_folder = f"{experimenter_folder}/{self.experiment_name.get()}"
        os.makedirs(experiment_folder, exist_ok=True)

        # Start reading from the devices
        self.device_mappings = get_device_mappings_by_usb_port()
        self.threads = []
        for mapping in self.device_mappings:
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            if not gpio_pins:
                continue
            q = self.port_queues[port_identifier]
            t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q))
            t.daemon = True
            t.start()
            self.threads.append(t)

    def update_gui(self):
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if isinstance(message, list):
                        # This is a data list from the FED3 device, save it for logging
                        self.save_to_csv(port_identifier, message)
                    elif message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass
        self.root.after(100, self.update_gui)

    def stop_experiment(self):
        stop_event.set()
        # Clean up threads and save the data
        for t in self.threads:
            t.join()  # Ensure all threads are properly stopped
        GPIO.cleanup()
        self.root.quit()  # Close the GUI window properly

    def save_to_csv(self, port_identifier, data_row):
        experimenter_folder = f"./{self.experimenter_name.get()}"
        experiment_folder = f"{experimenter_folder}/{self.experiment_name.get()}"
        os.makedirs(experiment_folder, exist_ok=True)
        
        filename = f"{experiment_folder}/{port_identifier}.csv"
        file_exists = os.path.isfile(filename)
        with open(filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            if not file_exists:
                writer.writerow(column_headers)  # Write header only once
            writer.writerow(data_row)

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()
